In [320]:
import pandas as pd

In [321]:
df = pandas.read_csv(r'hackaturing.dsv', sep='|')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [322]:
df.rename(columns={'base_hackaturing.cnpj':'cnpj', 
                          'base_hackaturing.prestador':'prestador',
                          'base_hackaturing.uf':'uf',
                          'base_hackaturing.id_beneficiario':'id_beneficiario',
                          'base_hackaturing.sexo':'sexo',
                          'base_hackaturing.data_nascimento':'data_nascimento',
                          'base_hackaturing.id_conta':'id_conta',
                          'base_hackaturing.cid':'cid',
                          'base_hackaturing.crm_solicitante':'crm_solicitante',
                          'base_hackaturing.cbos_solicitante':'cbos_solicitante',
                          'base_hackaturing.cbos_executante':'cbos_executante',
                          'base_hackaturing.data_entrada':'data_entrada',
                          'base_hackaturing.data_saida':'data_saida',
                          'base_hackaturing.data_item':'data_item',
                          'base_hackaturing.senha':'senha',
                          'base_hackaturing.tipo_guia':'tipo_guia',
                          'base_hackaturing.tipo_item':'tipo_item',
                          'base_hackaturing.carater_atendimento':'carater_atendimento',
                          'base_hackaturing.servico':'servico',
                          'base_hackaturing.descricao_despesa':'descricao_despesa',
                          'base_hackaturing.quantidade':'quantidade',
                          'base_hackaturing.valor_item':'valor_item',
                          'base_hackaturing.valor_cobrado':'valor_cobrado',
                          'base_hackaturing.valor_pago':'valor_pago',
                          'base_hackaturing.ano_mes':'ano_mes'}, inplace=True)

In [323]:
#todos os registros que tem algum valor cobrado (em torno de 600k)
df_glosa = df[df.valor_cobrado > 0]

#todos os registros nos quais ocorrem glosa (em torno de 12k)
df_glosa2 = df_glosa[(abs(df_glosa['valor_cobrado'] - df_glosa['valor_pago']) > 0)]
df_glosa2 = df_glosa2[0::1]

#uma amostra dos registros onde nao occore glosa (em torno de 12k)
df_glosa3 = df_glosa[(abs(df_glosa['valor_cobrado'] - df_glosa['valor_pago']) == 0)]
df_glosa3 = df_glosa3[0::51]

#definindo um "ground truth"
df_glosa3['truth'] = 0
df_glosa2['truth'] = 1

In [324]:
#adicionar os elementos
data_pretraining = df_glosa2
data_pretraining = data_pretraining.append(df_glosa3)

#embaralhar os elementos
data_pretraining = data_pretraining.sample(frac=1)

In [325]:
#parsear os parametros para os dados finais
data_training = pd.get_dummies(data_pretraining['cnpj'])
data_training = pd.concat([data_training, pd.get_dummies(data_pretraining['sexo'])], axis=1)
#data_training = pd.concat([data_training, pd.to_datetime(data_pretraining['data_nascimento'])], axis=1)
#data_training = pd.concat([data_training, (pd.to_datetime(data_pretraining['data_saida']) - pd.to_datetime(data_pretraining['data_entrada'])).dt.days], axis=1)
data_training = pd.concat([data_training, pd.get_dummies(data_pretraining['tipo_guia'])], axis=1)
data_training = pd.concat([data_training, pd.get_dummies(data_pretraining['tipo_item'])], axis=1)
data_training = pd.concat([data_training, pd.get_dummies(data_pretraining['carater_atendimento'])], axis=1)
#data_training = pd.concat([data_training, pd.get_dummies(data_pretraining['servico'])], axis=1) #enorme
data_training = pd.concat([data_training, data_pretraining['quantidade']], axis=1)
data_training = pd.concat([data_training, data_pretraining['valor_item']], axis=1)
data_training = pd.concat([data_training, data_pretraining['valor_cobrado']], axis=1)

data_target = data_pretraining['truth']

In [326]:
#treinar
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(data_training[:20], data_target[:20])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [327]:
clf.predict(data_training[:20])

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0])

In [328]:
data_target[:20]

2804952    1
2831079    0
2037393    1
221600     1
841138     0
4686527    0
3004786    0
2430732    0
368668     1
225935     1
2814993    1
2662004    1
2922330    0
2968639    1
4542123    0
4343598    1
2230558    0
2504163    0
3942400    1
2741641    0
Name: truth, dtype: int64